# Microgrid sizing optimization

Following the first example about [Microgrid performance simulation](Microgrid_Wind-Solar.ipynb)
using [Microgrids.jl](https://github.com/Microgrids-X/Microgrids.jl),
this notebook delves into **optimal microgrid sizing**.
This means finding the optimal size (rated power or energy) of each component by solving an optimization problem where the objective function is composed of Key Performance Indicators (KPIs) related to the economic and technical performance of the project.

Like in the previous example, the sizing is done for a microgrid project with *wind* and *solar* sources, plus a *battery* and a *dispatchable generator*.

<img alt='schematic of a microgrid with a symbolic representation of the notion of sizing' src='./images/microgrid_sizing - h2.png' style='height:18em'>

The main steps of the optimal sizing process are:
1. set up the problem (in particular loading project data like prices and time series)
2. run the optimization
3. analyze the results

## About the optimization formulation and process

### Optimization formulation

In this illustrative sizing example, the optimization objective (criterion) is:
- minimize the Levelized Costs of Electricity `lcoe` (€/kWh)
- while keeping the load shedding rate below some threshold (ex.: shedding rate ≤ 1%)

but other indicators could be used as well: rate of renewables above some threshold...

The above problem is a constrained optimization problem. Since we'll use unconstrained optimization solvers, we will reformulate it as: objective = lcoe + penalty for excess of shedding.

### Optimization process

This is a “blackbox” type of optimization problem: the optimization algorithm calls iteratively the Microgrid simulator with sizings to be tested until convergence (or reaching the maximum allowed number of iterations):

<img alt='schematic of microgrid sizing optimization process: iterations call of the simulator by optimizer' src='./images/MG_optim_diagram.png' style='height:18em'>

The code is meant for using the [NLopt](https://nlopt.readthedocs.io/en/latest/) library of optimization solvers (through its [NLopt.jl](https://github.com/JuliaOpt/NLopt.jl/) wrapper), but it could be adapted to other optimizers with minor changes (see remark below about the modularity of the code).

In [1]:
using Microgrids
using NLopt # optimization solvers
using Printf # pretty print results
using BenchmarkTools # for timing performance, optional

## Load Microgrid project data

Loading parameters and time series for a Microgrid project with *wind* and *solar* sources, plus a *battery* and a *dispatchable generator*. 
Values gathered from the [Microgrid_Wind-Solar.ipynb]((Microgrid_Wind-Solar.ipynb)) notebook.

In [2]:
include("./Microgrid_Wind-Solar_data.jl");

Data definition for Microgrid with wind, solar, storage and generator...


## Setting up the cost function (criterion) to be optimized

The key coding objective is to **encapsulate** the microgrid simulators (`simulate` function of `Microgrids.jl` package) into an objective function that can be called by the optimization algorithm, that is which respects its expected interface (here NLopt).

To increase the modularity which facilites using optimization solvers others that NLopt's we implement the encapsulation by **3 nested functions**:

1. Simulation of Microgrid project described by a sizing vector `x` (vector input) → returns all simulation statistics
2. Extract KPIs of interest to build a multi-objective criterion: here lcoe and shedding rate
3. Combine these KPIs as one mono-objective criterion: here LCOE + penalty if shedding rate > shed_max
   - and match the interface expected by NLopt's optimizers

but if one cares more about compactness, this could be assembled into one single function.

### Wrapper of the Microgrid simulator

accept a sizing vector `x`, then create all `Microgrids.jl` components and call `simulate`

In [3]:
"""Simulate the performance of a Microgrid project of size `x`
with x=[power_rated_gen, energy_rated_sto, power_rated_pv, power_rated_wind]

Returns stats, costs
"""
    function simulate_microgrid(x)
    project = Project(lifetime, discount_rate, timestep, "€")
    # Split decision variables (converted MW → kW):
    power_rated_gen = x[1]*1000
    energy_rated_sto = x[2]*1000
    power_rated_pv = x[3]*1000
    power_rated_wind = x[4]*1000
    power_rated_elyz = x[5]*1000
    power_rated_fc = x[6]*1000
    capacity_rated_hytank = x[7]*1000
    

    # Create components
    gen = ProductionUnit(power_rated_gen,
        fuel_intercept, fuel_slope, fuel_price,
        investment_price_gen, om_price_gen, lifetime_gen,
        load_ratio_min_gen, replacement_price_ratio,
        salvage_price_ratio, input_unit_gen,output_unit_gen)
    ftank = Tank(capacity_rated_ftank,investment_price_ftank, om_price_ftank,lifetime_ftank,loss_factor_ftank,fuel_ini_ratio,
            fuel_min_ratio, fuel_max_ratio,replacement_price_ratio, salvage_price_ratio)
    fuel_cell = ProductionUnit(power_rated_fc,cons_intercept_fc, cons_rate_fc,cons_price_fc,investment_price_fc, om_price_fc,lifetime_fc,
                load_min_ratio_fc,replacement_price_ratio, salvage_price_ratio,input_unit_fc,output_unit_fc)
    hytank = Tank(capacity_rated_hytank,investment_price_hytank, om_price_hytank,lifetime_hytank,loss_factor_hytank,LoH_ini_ratio,
                LoH_min_ratio, LoH_max_ratio,replacement_price_ratio, salvage_price_ratio)
    dispatchables = DispatchableCompound{Float64}([gen,], [fuel_cell])
       tanks = TankCompound{Float64}(ftank,hytank)
   
    elyz = ProductionUnit(power_rated_elyz,cons_intercept_elyz,cons_slope_elyz,cons_price_elyz, investment_price_elyz, om_price_elyz, lifetime_elyz,
    load_min_ratio_elyz,replacement_price_ratio, salvage_price_ratio,input_unit_elyz,output_unit_elyz)

    batt = Battery(energy_rated_sto,
        investment_price_sto, om_price_sto, lifetime_sto, lifetime_cycles,
        charge_rate, discharge_rate, loss_factor_sto, SoC_min, SoC_ini,SoC_max,
        replacement_price_ratio, salvage_price_ratio)
    pv = Photovoltaic(power_rated_pv, irradiance,
        investment_price_pv, om_price_pv,
        lifetime_pv, derating_factor_pv,
        replacement_price_ratio, salvage_price_ratio)
    windgen = WindPower(power_rated_wind, cf_wind,
        investment_price_wind, om_price_wind,
        lifetime_wind,
        replacement_price_ratio, salvage_price_ratio)
    
   mg = Microgrid(project, Pload,dispatchables,
    [elyz,],tanks,batt, [
        pv,
        windgen
        ]
    )
    # Launch simulation:
    traj, stats, costs = simulate(mg)

    return mg, traj ,stats, costs
end

simulate_microgrid

In [4]:
capacity_rated_ftank

0.0

Test of the simulator wrapper (on a baseline sizing):

In [5]:
# Baseline sizing: same as in Microgrid_Wind-Solar.ipynb notebook
x_base = [power_rated_gen, energy_rated_sto, power_rated_pv,power_rated_wind, power_rated_elyz,power_rated_fc ,capacity_rated_hytank]/1000.
# run simulation:
traj, stats, costs = simulate_microgrid(x_base)
x_base, costs.lcoe, costs.npc/1e6

LoadError: type OperationStats has no field lcoe

### Build the objective functions (criteria)

- first bi-objective function x ↦ (lcoe, shedding rate)(x)
- then wrapped into a mono objective x ↦ J(x) by using a penalty for the excess of shedding rate
  - and match the interface expected by NLopt's optimizers

In [6]:
"Multi-objective criterion for microgrid performance: lcoe, shedding rate"
function obj_multi(x)
   mg, traj,stats, costs = simulate_microgrid(x)
    # Extract KPIs of interest
    lcoe = costs.lcoe # $/kWh
    shed_rate = stats.shed_rate; # in [0,1]
    return lcoe, shed_rate
end

obj_multi

In [7]:
"""Mono-objective criterion: LCOE + penalty if shedding rate > `shed_max`

with signature adapted to NLopt with `grad` as 2nd argument

load shedding penalty threshold `shed_max` should be in  [0,1[
"""
function obj(x, grad, shed_max, w_shed_max=1e5)
    lcoe, shed_rate = obj_multi(x)
    over_shed = shed_rate - shed_max
    if over_shed > 0.0
        penalty = w_shed_max*over_shed
    else
        penalty = 0.0
    end
    J = lcoe + penalty
end

obj

### Tests the objective functions

Sizing being tested:
- baseline sizing from the simulation notebook: perfect quality of service (QoS) with zero load shedding
- baseline modified with a halved generator sizing: very good QoS with a bit of load shedding → not penalized
- small PV and small wind generators only: low LCOE (i.e. the production-only LCOE of these sources) but but extremely bad QoS → huge penalty

In [8]:
# Test:
shed_max = 0.01 # in [0,1]

x_shed = [0, energy_rated_sto, power_rated_pv,power_rated_wind, power_rated_elyz,power_rated_fc ,capacity_rated_hytank]/1000.
x_pv   = [0. 0. 500.   0. 0. 0. 0.]/1000.
x_wind = [0. 0.   0. 500. 0. 0. 0.]/1000.

println("Base. multi: ", obj_multi(x_base), " mono: ", obj(x_base,[], shed_max))
println("Shed. multi: ", obj_multi(x_shed), " mono: ", obj(x_shed,[], shed_max))
println("PV.   multi: ", obj_multi(x_pv), " mono: ", obj(x_pv,[], shed_max))
println("Wind. multi: ", obj_multi(x_wind), " mono: ", obj(x_wind,[], shed_max))

Base. multi: (0.21571461172077722, 0.20552617597582035) mono: 19552.833312193754
Shed. multi: (0.20827574936977203, 0.20552617597582035) mono: 19552.825873331403
PV.   multi: (0.10149685980966677, 0.923547868561659) mono: 91354.8883530257
Wind. multi: (0.10040264224635914, 0.74395737102815) mono: 73395.83750545724


## Optimization

### Setting up the optimization problem

bounds of the design space and starting point: derived from maximal load power

In [9]:
Pload_max = maximum(Pload)

xmin = [0., 0., 1e-3, 0., 0., 0., 0.] # 1e-3 instead of 0.0, because LCOE is NaN if ther is exactly zero generation
x0 = [0., 3.0, 3.0, 2.0, 1.0, 1.0, 5.0] * (Pload_max/1000)
xmax = [0., 10.0, 15.0, 5.0, 20., 1.2 , 10. ] * (Pload_max/1000)

7-element Vector{Float64}:
  0.0
 17.07
 25.605
  8.535
 34.14
  2.0484
 17.07

Optionally ban some choices:

In [10]:
# Solar power forbidden (optional)
#x0[3] = 1e-3
#xmax[3] = 1e-3
# Wind power forbidden (optional)
#x0[4] = 0.
#xmax[4] = 0.

Check cost function on `xmin` and `xmax`

In [11]:
obj_multi(xmin), obj_multi(xmax)

((0.10149685980963595, 0.9998470957371233), (1.2761120605599305, 8.262599201630023e-6))

### Wrapper of the optimization process

This is an optional step, but recommended to explore easily the impact of the many parameters taken by optimization algorithms.

See [optimization termination conditions](https://nlopt.readthedocs.io/en/latest/NLopt_Introduction/#termination-conditions) in NLopt documention for the meaning of `xtol_rel`

In [12]:
"""Optimize sizing of microgrid based on the `obj` function

Parameters:
- `x0`: initial sizing (for the algorithms which need them)
- `shed_max`: load shedding penalty threshold (same as in `obj`)
- `algo` could be one of LN_SBPLX, GN_DIRECT, GN_ESCH...
- `maxeval`: maximum allowed number of calls to the objective function,
  that is to the microgrid simulation
- `xtol_rel`: termination condition based on relative change of sizing, see NLopt doc.
- `srand`: random number generation seed (for algorithms which use some stochastic search)

Problem bounds are taken as the global variables `xmin`, `xmax`,
but could be added to the parameters as well.
"""
function optim_mg(x0, shed_max, algo=:LN_SBPLX, maxeval=1000, xtol_rel=1e-4, srand=1)
    nx = length(x0) # number of optim variables
    opt = Opt(algo, nx)
    NLopt.srand(srand)
    
    opt.lower_bounds = xmin
    opt.upper_bounds = xmax
    opt.min_objective = (x, grad) -> obj(x, grad, shed_max)
    opt.xtol_rel = xtol_rel
    opt.maxeval = maxeval
    
    (fopt, xopt, ret) = optimize(opt, x0)
    return xopt, ret, opt.numevals
end

optim_mg

### Run optimization & analyze results

In [13]:
algo = :GN_CRS2_LM # could be one of GN_CRS2_LM, GN_DIRECT, GN_ESCH, LN_SBPLX...
shed_max = 0. # in [0,1]
maxeval=10000
xopt, ret, numevals = optim_mg(x0, shed_max, algo, maxeval)

@printf("%s algo: %s after %d iterations. \nx*=", algo, ret, numevals)
println(round.(xopt*1000; digits=1)) # kW
lcoe_opt, shed_rate_opt = obj_multi(xopt)
println("LCOE*: ", lcoe_opt)
println("shed*: ", shed_rate_opt)

GN_CRS2_LM algo: MAXEVAL_REACHED after 10000 iterations. 
x*=[0.0, 9764.2, 20727.4, 3356.4, 380.0, 1541.9, 16800.5]
LCOE*: 0.5786213467955624
shed*: 0.0


In [24]:
algo = :GN_CRS2_LM # could be one of GN_CRS2_LM, GN_DIRECT, GN_ESCH, LN_SBPLX...
shed_max = 0.001 # in [0,1]
maxeval=10000
xopt, ret, numevals = optim_mg(x0, shed_max, algo, maxeval,1e-10,5)

@printf("%s algo: %s after %d iterations. \nx*=", algo, ret, numevals)
println(round.(xopt*1000; digits=1)) # kW
lcoe_opt, shed_rate_opt = obj_multi(xopt)
println("LCOE*: ", lcoe_opt)
println("shed*: ", shed_rate_opt)

GN_CRS2_LM algo: MAXEVAL_REACHED after 10000 iterations. 
x*=[0.0, 3402.7, 2074.2, 2217.4, 1196.7, 1301.9, 16800.9]
LCOE*: 0.20798206778104475
shed*: 0.0009998209597289068


optional local "polishing":

In [25]:
algo_polish = :LN_SBPLX
xopt_polish, ret, numevals = optim_mg(xopt, shed_max, algo_polish)

@printf("%s polish: %s after %d iterations. \nx*=", algo_polish, ret, numevals)
println(round.(xopt_polish*1000; digits=1)) # kW
lcoe_opt, shed_rate_opt = obj_multi(xopt_polish)
println("LCOE*: ", lcoe_opt)
println("shed*: ", shed_rate_opt)

LN_SBPLX polish: XTOL_REACHED after 239 iterations. 
x*=[0.0, 3402.5, 2074.2, 2217.4, 1196.6, 1301.8, 16801.0]
LCOE*: 0.20798046348779503
shed*: 0.000999888188853352


Retrieve all performance statistics of the optimized sizing

In [15]:
traj_opt, stats_opt, costs_opt = simulate_microgrid(xopt_polish)
@printf("NPC*: %.2f M\$ (compared to %.2f M\$ in baseline)\n", costs_opt.npc/1e6, costs.npc/1e6)
@printf("rate of renewables: %.1f %%\n", stats_opt.renew_rate*100)
# Display all operation statistics
stats_opt

LoadError: type OperationStats has no field npc

#### Summary of some results for 1% max shedding rate

Global search algorithms:
- **CRS2**: 0.190 720 \$/kWh (with shedding ~ threshold).
  x* = [889.0, 2013.0, 1570.9, 1354.0] → *best global algorithm choice so far, although effect of seed not studied*
- **DIRECT**: 0.191 297 \$/kWh (with shedding < threshold) in 1000 iter.
  x* = [910.4, 1837.8, 1543.3, 1375.7] 
    - "polishing" with SBPLX: 0.190 650 \$/kWh (with shedding ~ threshold) in 207 extra iter.
      x* = [887.1, 1992.4, 1543.3, 1375.6]
    - increasing DIRECT maxeval to 10k doesn't bring much improvement (0.191 258 \$/kWh)
      compared to SBPLX polishing
- **ESCH**: 0.194 884 \$/kWh with shedding OK (with shedding < threshold) in 1000 iter.
  x* = [1028.1, 1769.6, 1420.5, 1365.1] → *much slower to converge than DIRECT or CRS*

Local search algorithms:
- **SUBPLX**: 0.190 630 \$/kWh with (with shedding ~ threshold) **in 479 iter** (<maxeval).
  x* = [888.6, 1873.5, 1558.5, 1355.9]
  - very competitive, but too sensitive to initialization `x0`?

#### Case with perfect quality of service (QoS)

Another optimization case with perfect quality of service (QoS), that is zero load shedding  

In [ ]:
algo = :GN_CRS2_LM # could be one of LN_SBPLX, GN_DIRECT, GN_CRS2_LM, GN_ESCH...
shed_max = 0.00 # in [0,1]
maxeval=1000
xopt, ret, numevals = optim_mg(x0, shed_max, algo, maxeval)

@printf("%s algo: %s after %d iterations. \nx*=", algo, ret, numevals)
println(round.(xopt*1000; digits=1)) # kW
lcoe_opt, shed_rate_opt = obj_multi(xopt)
println("LCOE*: ", lcoe_opt)
println("shed*: ", shed_rate_opt)

In [14]:
algo_polish = :LN_SBPLX
xopt_polish, ret, numevals = optim_mg(xopt, shed_max, algo_polish)

@printf("%s polish: %s after %d iterations. \nx*=", algo_polish, ret, numevals)
println(round.(xopt_polish*1000; digits=1)) # kW
lcoe_opt, shed_rate_opt = obj_multi(xopt_polish)
println("LCOE*: ", lcoe_opt)
println("shed*: ", shed_rate_opt)


LN_SBPLX polish: XTOL_REACHED after 477 iterations. 
x*=[0.0, 16138.7, 9004.4, 6778.6, 507.3, 1398.0, 1994.7]
LCOE*: 0.6114079791076036
shed*: 0.0


LoadError: UndefVarError: `costs_opt` not defined

In [19]:
mg, traj, stats, costs = simulate_microgrid(xopt)
stats

OperationStats with fields:
- served_energy: 6.775e6 kWh
- shed_energy: 0.0 kWh
- shed_max: 0.0 kW
- shed_hours: 0.0 h
- shed_duration_max: 0.0 h
- shed_rate: 0.0 in [0,1]
- gen_energy: 0.0 kWh
- gen_hours: 0.0 h
- gen_fuel: 0.0 L
- fc_energy: 103580.0 kWh
- fc_hours: 143.0 h
- h2_consumed: 6473.8 kg
- h2_produced: 6954.6 kg
- elyz_hours: 3920.0 h
- elyz_consumed_energy: 389460.0 kWh
- h2_chain_loss: 285850.0 kWh
- storage_cycles: 41.344 
- storage_char_energy: 726430.0 kWh
- storage_dis_energy: 673370.0 kWh
- storage_loss_energy: 69990.0 kWh
- spilled_energy: 1.7264e7 kWh
- spilled_max: 8847.8 kW
- spilled_rate: 0.70818 in [0,1]
- renew_potential_energy: 2.4378e7 kWh
- renew_energy: 7.1139e6 kWh
- renew_rate: 1.0 in [0,1]


In [29]:
costs.npc/1e6

58.38101140623111

In [21]:
using Printf, CSV, DataFrames, Statistics
using PyPlot
with_plots = isdefined(Main, :PyPlot)
td = collect((0:nsteps-1)/24) # time in days

function plot_oper_traj(microgrid, oper_traj)
    fig, (ax1, ax2) = plt.subplots(2,1, sharex=true)
    ax1.plot(td, Pload, label="load")
    ax1.plot(td, oper_traj.Pfc, "tab:red", label="Pfc")
    ax1.plot(td, oper_traj.Prenew_pot, "tab:orange", label="ren")
    ax1.plot(td, oper_traj.Pelyz, "tab:gray", label="Pelyz")
    ax1.plot(td, oper_traj.Pfc, "tab:pink", label="Pfc")
     ax3=ax1.twinx()
     ax3.set_ylim([0, 1.25])
    ax3.plot(td, oper_traj.Ebatt[2:end]/(xopt[2]*1000), "C2")
    
    ax1.legend()
    ax1.grid(true)
    ax1.set(
        ylabel="kW"
    )
   
    ax2.grid(true)
    ax2.set(
        title="H2 Tank level",
        xlabel="time (d)",
        ylabel="kWh"
       
    )
    ax2.plot(td, oper_traj.LoH[1:end-1]/(xopt[7]*1000), "C2")
    
    fig.tight_layout()
    return fig, (ax1, ax2)
end

if with_plots
    fig, (ax1, ax2) = plot_oper_traj(mg, traj)
     ax1.set_ylim(0, 4250)
    pygui(true)
    plt.show()
   
end;

#### Performance (duration) of the optimization

About 2 s for 10k iterations, i.e. 0.2 ms/iter.

In [ ]:
@time xopt, ret, numevals = optim_mg(x0, shed_max, :GN_DIRECT, 10000)

Fact: optimization time relates to simulation time with Toptim ~ Tsim × neval

[Benchmarktools](https://juliaci.github.io/BenchmarkTools.jl/)'s `@btime` is need for accurate timing of the simulator:

In [15]:
@btime obj_multi(x0)

  50.694 ms (899212 allocations: 26.56 MiB)


(0.32398700745291287, 0.0002557645932088728)

## Optimization with Automatic Differentiation *(TO BE WRITTEN)*

As demonstrated in our conference article:

de Godoy Antunes, P. Haessig, C. Wang, and R. Chouhy Leborgne, “Optimal Microgrid Sizing using Gradient-based Algorithms with Automatic Differentiation,” ISGT Europe 2022, Novi Sad, Serbia, 2022. https://dx.doi.org/10.1109/ISGT-Europe54678.2022.9960498 https://hal.archives-ouvertes.fr/hal-03370004

Gradient of optimization criteria computed with [ForwardDiff.jl](https://juliadiff.org/ForwardDiff.jl/stable/).
This allows using **gradient-based optimization algorithms** like LBFGS or SLSQP (the latter supports constraints) which converge in less iterations.

This requires using the "relaxation parameter" `ε` of the Microgrid simulator to smooth out some discontinuities.

DOCUMENTATION TO BE WRITTEN

In [18]:
?aggregation

search: aggregation aggregate



```
aggregation(mg::Microgrid, oper_traj::OperationTraj, ε::Real=1.0)
```

Aggregate operation time series `oper_traj` into yearly statistics for the the microgrid `mg` (returned as an `OperationStats` object).

Discontinuous statistics can optionally be relaxed (smoothed) using the relaxation parameter `ε`:

  * 0.0 means no relaxation (default value)
  * 1.0 yields the strongest relaxation

Using relaxation (`ε` > 0) is recommended when using gradient-based optimization and then a “small enough” value between 0.05 and 0.30 is suggested.


---
## *How we imagine the optimization interface in a future version...*

Desirable feature: *dynamically select* which variables are optimized (e.g. using `getproperty` to dynamically access structure fields...)

In [19]:
#mg0 = baseline microgrid project, with given price and time series data
opti_params = [ # list of :component/variable name, initial, min, max values
    ([:generator,:power_rated], 1000.0, 0., 2000.), 
    ([:battery,:energy_max],    3000.0, 0., 6000.)
]
#optim_mg(mg0, opti_params)

2-element Vector{Tuple{Vector{Symbol}, Float64, Float64, Float64}}:
 ([:generator, :power_rated], 1000.0, 0.0, 2000.0)
 ([:battery, :energy_max], 3000.0, 0.0, 6000.0)